In [1]:
#based on https://stackoverflow.com/questions/43101497/how-do-stream-data-to-a-bokeh-plot-in-jupyter-with-a-high-refresh-rate

import time
import os
import signal
from threading import Thread, Event

from sh import tail, cat

import numpy as np
import pmt

from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.models import ColumnDataSource, Range1d, LinearAxis
from bokeh.io import output_notebook, reset_output, show, push_notebook

from ktdashboard.ktdashboard import KTdashboard
 
# reset_output()
output_notebook()


Loading BokehJS ...

In [2]:
height = 500
width = 800
WITH_KT = False

fig_power = figure(
        height=height,
        width=width,
        title="PowerSensor3 measurements",
        y_range=(0, 12)
    )
fig_power.xaxis.axis_label = "Time (s)"
fig_power.yaxis.axis_label = "Power (W)"

fig_gpu = figure(
        height=height,
        width=width,
        title="GPU status",
        y_range=(0, 1000)
    )
fig_gpu.xaxis.axis_label = "Time (s)"
fig_gpu.yaxis.axis_label = "GPU Frequency (MHz)"
fig_gpu.extra_y_ranges = {"load": Range1d(start=0, end=100)}
fig_gpu.add_layout(LinearAxis(y_range_name="load", axis_label="GPU load (%)"), 'right')

data = ColumnDataSource(data={'time': [], 'power_ps': [], 'power_tegra_cpu': [], 'power_tegra_gpu': [], 'gpu_freq': [], 'gpu_load': []})

line_ps = fig_power.line("time", "power_ps", source=data, legend_label="PowerSensor3", line_width=2)
line_tegra_cpu = fig_power.line("time", "power_tegra_cpu", source=data, color='red', legend_label="Tegra - CPU", line_width=2)
line_tegra_gpu = fig_power.line("time", "power_tegra_gpu", source=data, color='green', legend_label="Tegra - GPU", line_width=2)

line_gpu_freq = fig_gpu.line("time", "gpu_freq", source=data, legend_label="Frequency", line_width=2)
line_gpu_load = fig_gpu.line("time", "gpu_load", source=data, color='red', legend_label="Load", line_width=1, y_range_name="load")

fig_power.legend.location = "top_left"
fig_gpu.legend.location = "top_left"

if WITH_KT:
    # add the kernel tuner dashboard, with a custom figure
    ktdash = KTdashboard("/home/powersensor/PowerSensor/powersensor_demo/dedisp_cache.json", demo=False)
    fig_kt = figure(
            height=height,
            width=width,
            title="Kernel Tuner"
    )
    
    kt_xvariable = "GB/s/W (system)"
    kt_yvariable = "GB/s"
    kt_colorvariable = "GPU frequency (MHz)"
    fig_kt.xaxis.axis_label = kt_xvariable
    fig_kt.yaxis.axis_label = kt_yvariable
    fig_kt.scatter(kt_xvariable, kt_yvariable, size=12, color=ktdash.update_colors(kt_colorvariable), alpha=.5, source=ktdash.source)

if WITH_KT:
    handle = show(row(fig_power, fig_gpu, fig_kt), notebook_handle=True)
else:
    handle = show(row(fig_power, fig_gpu), notebook_handle=True)

In [3]:
event = Event()

def update_sensors(_, event):
    n_sec_to_show = 30
    # powersensor dumpefile freq reduced by a factor 2000: so 10 Hz
    ps_freq = 10
    n_show = n_sec_to_show * ps_freq

    tegra = pmt.get_pmt('tegra')
    tegra.read()

    # wait for file to exist
    fname = '/dev/shm/ps3.txt'
    timeout = 9999  # seconds
    tstart = time.time()
    while not os.path.exists(fname):
        event.wait(.1)
        if time.time() - tstart > timeout:
            print(f"File {fname} still does not exist, giving up")
            return

    print("Starting read")

    process = tail('-F', '-n 0', '/dev/shm/ps3.txt', _iter=True)
    # first_offset = None
    for line in process:
        if not line.startswith('S'):
            continue

        line = line.split(' ')
        t = float(line[1])
        p = float(line[-1])

        # if first_offset is None:
            # first_offset = time.time() - t

        # offset = time.time() - t

        # warn if we're more than 50ms out of sync
        # if offset - first_offset > .05:
        #     print(f"Offset large! {offset - first_offset=}")
        #     event.set()

        gpu_freq = int(cat('/sys/class/devfreq/57000000.gpu/cur_freq')) // 1000000
        gpu_load = int(cat('/sys/class/devfreq/57000000.gpu/device/load')) / 10

        state = tegra.read()
        power_tegra_gpu = state.watts(2)
        power_tegra_cpu = state.watts(3)
        
        data.stream({'time': [t], 'power_ps': [p], 'power_tegra_gpu': [power_tegra_gpu], 'power_tegra_cpu': [power_tegra_cpu], 'gpu_freq': [gpu_freq], 'gpu_load': [gpu_load]}, n_show)
        
        if event.is_set():
            # print(offset)
            process.signal(signal.SIGINT)
            return
            
    print("Sensor update stopped")

def update_figs(id, event):
    while not event.is_set():
        push_notebook(handle=handle)
        event.wait(.1)
    print("Figure update stopped")

def update_kt(id, event):
    while not event.is_set():
        ktdash.update_data()
        event.wait(5)
    print("Kernel tuner update stopped")
        


if WITH_KT:
    funcs = (update_sensors, update_kt, update_figs)
else:
    funcs = (update_sensors, update_figs)

threads = []
for func in funcs:
    threads.append(Thread(target=func, args=(id, event)))

for t in threads:
    t.start()

Starting read


In [5]:
# event.set()